### Setup

In [3]:
import pandas as pd
import requests

# Utils
import json
from pandas import json_normalize
import enum

# Options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

headers = {'Content-Type': 'application/json'}

In [38]:
import os
from google.cloud import secretmanager
#secrets = secretmanager.SecretManagerServiceClient()

#resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
#secret = secrets.access_secret_version(name=resource_name)
#data = secret.payload.data.decode('UTF-8')

### Config

In [4]:
class TeamType(enum.Enum):
    PRODUCT = "Produktteam"
    IT = "IT-team"
    ADMINISTRATION = "Forvaltningsteam"
    PROJECT = "Prosjektteam"
    OTHER = "Annet"
    UNKNOWN = "Ukjent"

In [5]:
class AreaType(enum.Enum):
    IT = "IT-område"
    PRODUCT_AREA = "Produktområde"
    PROJECT = "Prosjekt"
    OTHER = "Annet"

### Get Data

In [6]:
df_productarea = pd.DataFrame()
url = 'https://teamkatalog-api.nais.adeo.no/productarea'
r = requests.get(url, headers=headers)
content = r.json()['content']
normalized = json_normalize(content)
df_productarea = df_productarea.append(normalized, sort=True)
df_productarea['member-list'] = df_productarea['members'].apply(lambda x: json_normalize(x).to_records())

/tmp/ipykernel_68/1016234608.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_productarea = df_productarea.append(normalized, sort=True)


In [16]:
df_productarea.head(1)

,areaType,changeStamp_lastModifiedDate,defaultArea,description,id,links_slackChannels,links_ui,members,name,paOwnerGroup,paOwnerGroup_ownerGroupMemberResourceList,paOwnerGroup_ownerResource,paOwnerGroup_ownerResource_email,paOwnerGroup_ownerResource_endDate,paOwnerGroup_ownerResource_familyName,paOwnerGroup_ownerResource_fullName,paOwnerGroup_ownerResource_givenName,paOwnerGroup_ownerResource_links_ui,paOwnerGroup_ownerResource_navIdent,paOwnerGroup_ownerResource_onLeave,paOwnerGroup_ownerResource_resourceType,paOwnerGroup_ownerResource_stale,paOwnerGroup_ownerResource_startDate,slackChannel,status,tags,member-list
0,Produktområde,2022-05-07T11:48:31.0226,False,I produktområde helse utvikler vi digitale tje...,6b149078-927b-4570-a1ce-97bbb9499fb6,"[{'name': '#område-helse', 'url': 'https://sla...",https://teamkatalog.nav.no/area/6b149078-927b-...,"[{'navIdent': 'M164897', 'description': None, ...",Produktområde helse,NaN,"[{'navIdent': 'W138575', 'givenName': 'Kristin...",NaN,Kristian.Munthe@nav.no,NaN,Munthe,Kristian Munthe,Kristian,https://teamkatalog.nav.no/resource/M106440,M106440,False,INTERNAL,False,2000-12-28,#område-helse,ACTIVE,[],"[[0, M164897, None, [AREA_LEAD], 0, None, None..."


In [8]:
df_productarea = df_productarea.drop(columns=['changeStamp.lastModifiedBy'])

In [20]:
df_productarea.dtypes

areaType                                      object
changeStamp_lastModifiedDate                  object
defaultArea                                     bool
description                                   object
id                                            object
links_slackChannels                           object
links_ui                                      object
members                                       object
name                                          object
paOwnerGroup                                 float64
paOwnerGroup_ownerGroupMemberResourceList     object
paOwnerGroup_ownerResource                   float64
paOwnerGroup_ownerResource_email              object
paOwnerGroup_ownerResource_endDate           float64
paOwnerGroup_ownerResource_familyName         object
paOwnerGroup_ownerResource_fullName           object
paOwnerGroup_ownerResource_givenName          object
paOwnerGroup_ownerResource_links_ui           object
paOwnerGroup_ownerResource_navIdent           

In [11]:
def convert_area_type(area_type):
    if area_type:
        return AreaType[area_type].value
    else:
        return AreaType.OTHER.value

In [12]:
df_productarea["areaType"] = df_productarea["areaType"].apply(convert_area_type)

In [15]:
df_productarea.columns = df_productarea.columns.str.replace('.', '_')

/tmp/ipykernel_68/3703384248.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_productarea.columns = df_productarea.columns.str.replace('.', '_')


In [17]:
df_teams = pd.DataFrame()
url = 'https://teamkatalog-api.nais.adeo.no/team'
r = requests.get(url, headers=headers)
content = r.json()['content']
normalized = json_normalize(content)
df_teams = df_teams.append(normalized, sort=True)
df_teams['member-list'] = df_teams['members'].apply(lambda x: json_normalize(x).to_records())

/tmp/ipykernel_68/1674685619.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_teams = df_teams.append(normalized, sort=True)


In [26]:
df_teams.head(1)

,changeStamp_lastModifiedDate,clusterIds,contactAddresses,contactPersonIdent,description,id,links_slackChannels,links_ui,members,naisTeams,name,officeHours,officeHours_days,officeHours_information,officeHours_location_code,officeHours_location_description,officeHours_location_displayName,officeHours_location_parent_code,officeHours_location_parent_description,officeHours_location_parent_displayName,officeHours_location_parent_parent_code,officeHours_location_parent_parent_description,officeHours_location_parent_parent_displayName,officeHours_location_parent_parent_parent,officeHours_location_parent_parent_type,officeHours_location_parent_type,officeHours_location_type,productAreaId,qaTime,slackChannel,status,tags,teamOwnerIdent,teamOwnershipType,teamType,member-list
0,2023-02-06T12:02:05.039352,[],"[{'address': 'CSV5ZTXCH', 'type': 'SLACK'}]",G165874,Vi utvikler en modul som fanger opp saker der ...,0dd4d621-3482-4600-a356-e49666f1f1f8,"[{'name': '#team-risk-værsågod', 'url': 'https...",https://teamkatalog.nav.no/team/0dd4d621-3482-...,"[{'navIdent': 'G153965', 'description': None, ...",[risk],Team Risk,NaN,[],Vi er et Remote-first-team. Samlingsbasert.,FA1-BB-E6,Etasje 6,"Fyrstikkalléen 1, Bygg B, Etasje 6",FA1-BB,Bygg B,"Fyrstikkalléen 1, Bygg B",FA1,Fyrstikkalléen 1,Fyrstikkalléen 1,NaN,BUILDING,SECTION,FLOOR,6b149078-927b-4570-a1ce-97bbb9499fb6,2020-08-31T13:35:45.161548,team-risk-værsågod,ACTIVE,[],None,PRODUCT,COMPLICATED_SUBSYSTEM,"[[0, G153965, None, [DEVELOPER, SECURITY_CHAMP..."


In [25]:
df_teams.dtypes

changeStamp_lastModifiedDate                       object
clusterIds                                         object
contactAddresses                                   object
contactPersonIdent                                 object
description                                        object
id                                                 object
links_slackChannels                                object
links_ui                                           object
members                                            object
naisTeams                                          object
name                                               object
officeHours                                       float64
officeHours_days                                   object
officeHours_information                            object
officeHours_location_code                          object
officeHours_location_description                   object
officeHours_location_displayName                   object
officeHours_lo

In [21]:
df_teams = df_teams.drop(columns=['changeStamp.lastModifiedBy'])

In [22]:
def convert_team_type(team_type):
    return TeamType[team_type].value

In [24]:
df_teams.columns = df_teams.columns.str.replace('.', '_')

/tmp/ipykernel_68/2535767319.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_teams.columns = df_teams.columns.str.replace('.', '_')


### Publish data to BigQuery

In [29]:
#Navn på prosjekt og dataset som definert i BigQuery
project_id = 'org-prod-1016'
dataset = 'teamkatalogen'

from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [30]:
df_teams_pub= df_teams.copy()

In [31]:
#df_teams_pub = df_teams_pub.astype({ 'tagger': 'string','regelverk': 'string', 'suksesskriterier': 'string', 'relevansFor': 'string'})

In [32]:
#Istedenfor å spesifisere typen på variablene så plukker vi ut én kolonne til å teste med.
df_teams_pub = df_teams_pub[['name']]

In [40]:
table = 'teams'
destination_table = f"{project_id}.{dataset}.{table}"

job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE",)
job = client.load_table_from_dataframe(df_teams_pub, destination_table, job_config=job_config)
job.result()

Forbidden: 403 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/org-prod-1016/jobs?uploadType=multipart: Access Denied: Project org-prod-1016: User does not have bigquery.jobs.create permission in project org-prod-1016.